#  思路
1. 构造一个class BasicExp，生成基本的四则运算表达式Exp: 输入参数必选：两个操作数（num1, num2），一个操作符(op)构成; 可选：是否带括号（bracelet）;
2. op若不指定，默认选取范围在四个操作符中['+', '-', '*', '/',] 随机选取。
3. bracelet = True 或者 False，决定生成的Exp是否带括号。
4. 默认num1, num2的生成方法：num1,num2 若未指定，默认选取范围【0，100】，随机生成，可以是整数，或者小数；num2要求总是大于等于num1；
5. 根据 op 的不同，预留重载四种num1,num2的生成方法
6. 一个子函数生成Exp： f"({num1} {op} {num2})" if bracelet else f"{num1} {op} {num2}"
7. 一个子函数调用Exp，使用eval()方法得到计算结果

In [10]:
import random
debug = False
# 占位符
# NULL_4 = '\u0000' * 4
# NULL_8 = '\u0000' * 8
NULL_4 = ' ' * 4
NULL_8 = ' ' * 8

In [18]:

def generate_number(min_val=0, max_val=100, integer=True):
    return random.randint(min_val, max_val) if integer else random.uniform(min_val, max_val)

class BasicExp:
    def __init__(self, num1=None, num2=None, op=None, bracelet=False, integer=True):
        self.op = op if op is not None else random.choice(['+', '-', '*', '/'])
        self.bracelet = bracelet
        self.integer  = integer

        self.num1 = num1 if num1 is not None else generate_number(integer=self.integer)
        self.num2 = num2 if num2 is not None else generate_number(integer=self.integer)

        self.op_str = self.op
        self.exp = None

    def generate_exp(self):
        op_str = self.op
        if self.op == '+':
            self.num1, self.num2 = self.generate_addition_numbers()
        elif self.op == '-':
            self.num1, self.num2 = self.generate_subtraction_numbers()
        elif self.op == '*':
            self.num1, self.num2 = self.generate_multiplication_numbers()
            self.op_str = 'x'
        elif self.op == '/':
            self.num1, self.num2 = self.generate_division_numbers()
            self.op_str = "\u00F7"

        self.exp     = f"({self.num1} {self.op} {self.num2})" if self.bracelet else f"{self.num1} {self.op} {self.num2}"
        self.exp_str = f"({self.num1} {self.op_str} {self.num2})" if self.bracelet else f"{self.num1} {self.op_str} {self.num2}"
        return self.exp_str

    def generate_addition_numbers(self):
        num1 = generate_number(integer=self.integer)
        num2 = generate_number(min_val=num1, max_val=100, integer=self.integer)
        return num1, num2

    def generate_subtraction_numbers(self):
        num2 = generate_number(integer=self.integer)
        num1 = generate_number(min_val=0, max_val=num2, integer=self.integer)
        return num1, num2

    def generate_multiplication_numbers(self):
        num1 = generate_number(integer=self.integer)
        num2 = generate_number(min_val=1, max_val=100, integer=self.integer)  # Avoid multiplying by zero
        return num1, num2

    def generate_division_numbers(self):
        num2 = generate_number(min_val=1, max_val=100, integer=self.integer)  # Avoid division by zero
        num1 = generate_number(min_val=0, max_val=100, integer=self.integer)  # Ensure num1 can be divided by num2
        return num1, num2
    
    def evaluate_exp(self):
        return eval(self.exp)

    def generateExpStr(self, ind:int):
        formatStr = f"[{ind:<2}] | {self.generate_exp():<10} = {NULL_4} | 答案：{self.evaluate_exp()}"
        if debug: print(formatStr)
        return formatStr


In [19]:
for i in range (1, 10):
    #exp = BasicExp(bracelet=random.randint(0,1), integer=1)
    exp = BasicExp(0, integer=1)
    # print(f"i={i}: Generated Expression: {exp.generate_exp()}")
    # print(f"i={i}: Evaluation Result   : {exp.evaluate_exp()}")
    print (exp.generateExpStr(i))
    print()

[1 ] | 39 + 96    =      | 答案：135

[2 ] | 17 + 83    =      | 答案：100

[3 ] | 43 - 91    =      | 答案：-48

[4 ] | 51 x 46    =      | 答案：2346

[5 ] | 32 - 42    =      | 答案：-10

[6 ] | 29 + 57    =      | 答案：86

[7 ] | 33 x 36    =      | 答案：1188

[8 ] | 17 + 49    =      | 答案：66

[9 ] | 2 - 24     =      | 答案：-22



In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdf_name = "multi_page_problem.pdf"

# have to use SimSun, otherwise can not generate pdf correctly
font_name = 'SimSun'
font_path = './SimSun.ttf'

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont(font_name, font_path))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName=font_name, name='Sun', leading=15, fontSize=13))  #leading=spacing
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        problem_text = gen_problem_per_day(nums, 'md_rnd', 70)
        lines = problem_text.split('\n')
        for line in lines:
            flowables.append(Paragraph(line, styles['Sun']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)

# 使用函数生成多页PDF
days = 3  # 
nums = 40  # 
print_problem_pdf(days, nums)